In [ ]:
import re
import torch
import unicodedata

from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

USE_CUDA = torch.cuda.is_available()

In [ ]:
import torch
import re
import unicodedata
from torch.utils.data import Dataset, DataLoader

# 유니코드 문자열을 아스키 문자열로 변환하는 함수
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# 문자열을 정규화하는 함수
def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([,.!?])", r" \1 ", s)
    s = re.sub(r"[^a-zA-Z,.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

class MNTDataset(Dataset):
    def __init__(self, txt_path, min_length, max_length):
        self.X, self.Y = [], []
        self.min_length = min_length
        self.max_length = max_length
        self.corpus = open(txt_path, 'r', encoding='utf-8').readlines()

        self.preprocess()
        self.build_vocab()

    def preprocess(self):
        for parallel in self.corpus:
            src, trg, _ = parallel.strip().split('\t')

            ## src 문장이나 trg 문장이 비어있는 경우 제외.
            if src.strip() == "" or trg.strip() == "":
                continue

            ## 문장을 정규화하고 단어 단위로 분리.
            normalized_src = normalize_string(src).split()
            normalized_trg = normalize_string(trg).split()

            if len(normalized_src) >= self.min_length and len(normalized_trg) <= self.max_length \
            and len(normalized_trg) >= self.min_length and len(normalized_src) <= self.max_length:
                self.X.append(normalized_src)
                self.Y.append(normalized_trg)

    def build_vocab(self):
        flatten = lambda l: [item for sublist in l for item in sublist]
        self.source_vocab = list(set(flatten(self.X)))
        self.target_vocab = list(set(flatten(self.Y)))
        print(len(self.source_vocab), len(self.target_vocab))

        self.source2index = {'<PAD>': 0, '<UNK>': 1, '<SOS>': 2, '<EOS>': 3}
        self.target2index = {'<PAD>': 0, '<UNK>': 1, '<SOS>': 2, '<EOS>': 3}

        for vocab in self.source_vocab:
            if self.source2index.get(vocab) is None: ## 단어가 사전에 없다면
                self.source2index[vocab] = len(self.source2index)
        self.index2source = {v:k for k, v in self.source2index.items()}

        for vocab in self.target_vocab:
            if self.target2index.get(vocab) is None:
                self.target2index[vocab] = len(self.target2index)
        self.index2target = {v:k for k, v in self.target2index.items()}

    def prepare_sequence(self, seq, to_index, max_len=None):
        idxs = list(map(lambda w: to_index[w] if to_index.get(w) is not None else to_index["<UNK>"], seq))
        if max_len is not None:
            # 패딩 추가
            idxs = idxs + [to_index['<PAD>']] * (max_len - len(idxs))
            idxs = idxs[:max_len]  # 최대 길이를 초과하지 않도록 자름
        return torch.LongTensor(idxs)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        src = self.X[idx]
        trg = self.Y[idx]
        return src, trg

    def collate_fn(self, batch):
        # 각 배치에서 소스와 타겟 시퀀스를 분리
        batch_src, batch_trg = zip(*batch)

        # 소스와 타겟 시퀀스의 최대 길이
        max_len_src = max([len(s) for s in batch_src]) + 1  # +1 for <EOS>
        max_len_trg = max([len(t) for t in batch_trg]) + 1  # +1 for <EOS>

        # 시퀀스를 인덱스로 변환하고 패딩을 추가
        src_sequences = [self.prepare_sequence(s + ['<EOS>'], self.source2index, max_len_src) for s in batch_src]
        trg_sequences = [self.prepare_sequence(t + ['<EOS>'], self.target2index, max_len_trg) for t in batch_trg]

        # 텐서로 결합
        src_batch = torch.stack(src_sequences)
        trg_batch = torch.stack(trg_sequences)

        # 각 시퀀스의 길이 계산
        src_lengths = [len(s) for s in batch_src]
        trg_lengths = [len(t) for t in batch_trg]

        return src_batch, trg_batch, src_lengths, trg_lengths

    def get_dataloader(self, batch_size, shuffle, num_workers):
        return DataLoader(self, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=self.collate_fn)



In [ ]:
dataset = MNTDataset("/home/pervinco/Datasets/fra-eng/fra.txt", 3, 25)

print(len(dataset.X), len(dataset.Y))
print(dataset.X[0], dataset.Y[0])

In [ ]:
dataloader = dataset.get_dataloader(256, True, 32)

for data in dataloader:
    print(len(data))
    print(data[0].shape)
    print(data[1].shape)
    print(data[2])
    print(data[3])

    print(data[0][0])

    break

## pack_padded_sequence, pad_packed_sequence

- ```torch.nn.utils.rnn.pack_padded_sequence``` : 패딩된 시퀀스를 입력 받아 패딩되지 않은 시퀀스로 변환.
- RNN 계열이 패딩된 부분을 처리하지 않도록 도와준다.
- 함수는 입력 시퀀스와 각 시퀀스의 실제 길이를 입력으로 받는다.

- ```pad_packed_sequence``` : 패딩되지 않은 시퀀스를 입력으로 받아 패딩된 시퀀스로 변환한다.
- 함수는 RNN 계열 모델의 출력을 처리할 때 사용되며 패딩된 시퀀스 형태로 복원해준다.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, n_layers=1, bidirec=False):
        super().__init__()

        self.input_size = input_size ## x_t의 차원
        self.hidden_size = hidden_size ## h_t의 차원
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, embedding_size)

        # 양방향 GRU를 사용할지 여부에 따라 GRU 레이어
        # 양방향 GRU를 사용하면, n_direction을 2로 설정하고, 그렇지 않으면 1로 설정
        if bidirec:
            self.n_direction = 2
            self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True, bidirectional=True)
        else:
            self.n_direction = 1
            self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True)

    # 은닉 상태를 초기화하는 함수
    # 입력의 크기에 따라 0으로 채워진 텐서를 생성하고, CUDA를 사용할 경우 GPU로 이동
    def init_hidden(self, inputs):
        hidden = Variable(torch.zeros(self.n_layers * self.n_direction, inputs.size(0), self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden

    # 가중치를 초기화하는 함수
    # 임베딩 레이어와 GRU 레이어의 가중치를 xavier_uniform 방식으로 초기화
    def init_weight(self):
        self.embedding.weight = nn.init.xavier_uniform(self.embedding.weight)
        self.gru.weight_hh_l0 = nn.init.xavier_uniform(self.gru.weight_hh_l0)
        self.gru.weight_ih_l0 = nn.init.xavier_uniform(self.gru.weight_ih_l0)

    # 순전파 정의
    # 입력과 입력 길이를 받아서 임베딩 레이어와 GRU 레이어를 통과
    # GRU 레이어의 출력과 은닉 상태를 반환
    def forward(self, inputs, input_lengths):
        hidden = self.init_hidden(inputs)

        embedded = self.embedding(inputs)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, batch_first=True)
        outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)

        # 레이어 수가 1보다 크면, 마지막 두 레이어의 은닉 상태를 사용하고, 그렇지 않으면 마지막 레이어의 은닉 상태를 사용
        if self.n_layers > 1:
            if self.n_direction == 2:
                hidden = hidden[-2:]
            else:
                hidden = hidden[-1]

        return outputs, torch.cat([h for h in hidden], 1).unsqueeze(1)


In [ ]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, n_layers=1, dropout_p=0.1):
        super().__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.dropout = nn.Dropout(dropout_p)

        # GRU 레이어를 생성. 임베딩 크기 + 은닉층 크기, 은닉층 크기, 레이어 수를 인자로 받음
        self.gru = nn.GRU(embedding_size + hidden_size, hidden_size, n_layers, batch_first=True)
        # 선형 레이어를 생성. 은닉층 크기 * 2, 입력 크기를 인자로 받음
        self.linear = nn.Linear(hidden_size * 2, input_size)
        # Attention 레이어를 생성. 은닉층 크기를 인자로 받음
        self.attn = nn.Linear(self.hidden_size, self.hidden_size)

    # 은닉 상태를 초기화하는 함수
    def init_hidden(self,inputs):
        hidden = Variable(torch.zeros(self.n_layers, inputs.size(0), self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden

    # 가중치를 초기화하는 함수
    def init_weight(self):
        self.embedding.weight = nn.init.xavier_uniform(self.embedding.weight)
        self.gru.weight_hh_l0 = nn.init.xavier_uniform(self.gru.weight_hh_l0)
        self.gru.weight_ih_l0 = nn.init.xavier_uniform(self.gru.weight_ih_l0)
        self.linear.weight = nn.init.xavier_uniform(self.linear.weight)
        self.attn.weight = nn.init.xavier_uniform(self.attn.weight)

    # Attention 메커니즘을 구현하는 함수
    def Attention(self, hidden, encoder_outputs, encoder_maskings):
        """
        hidden : 1,B,D
        encoder_outputs : B,T,D
        encoder_maskings : B,T # ByteTensor
        """
        # hidden의 차원을 변경
        hidden = hidden[0].unsqueeze(2)  # (1,B,D) -> (B,D,1)

        # encoder_outputs의 크기를 가져옴
        batch_size = encoder_outputs.size(0) # B
        max_len = encoder_outputs.size(1) # T
        # attention 에너지를 계산
        energies = self.attn(encoder_outputs.contiguous().view(batch_size * max_len, -1)) # B*T,D -> B*T,D
        energies = energies.view(batch_size,max_len, -1) # B,T,D
        attn_energies = energies.bmm(hidden).squeeze(2) # B,T,D * B,D,1 --> B,T

        # softmax를 사용하여 attention 가중치를 계산
        alpha = F.softmax(attn_energies,1) # B,T
        alpha = alpha.unsqueeze(1) # B,1,T
        # context 벡터를 계산
        context = alpha.bmm(encoder_outputs) # B,1,T * B,T,D => B,1,D

        return context, alpha

# 순전파 함수
    def forward(self, inputs, context, max_length, encoder_outputs, encoder_maskings=None, is_training=False):
        """
        inputs : B,1 (torch.LongTensor, 시작 심볼)
        context : B,1,D (FloatTensor, 마지막 인코더 은닉 상태)
        max_length : int, 디코딩할 최대 길이
        encoder_outputs : B,T,D
        encoder_maskings : B,T # ByteTensor
        is_training : bool, 드롭아웃을 훈련 단계에서만 적용하기 위함
        """

        embedded = self.embedding(inputs)
        hidden = self.init_hidden(inputs)

        if is_training:
            embedded = self.dropout(embedded)

        decode = []
        for i in range(max_length):
            # GRU의 입력으로 embedded와 context를 연결한 것을 사용
            _, hidden = self.gru(torch.cat((embedded, context), 2), hidden) # h_t = f(h_{t-1},y_{t-1},c)

            # hidden과 context를 연결하여 concated를 생성
            concated = torch.cat((hidden, context.transpose(0, 1)), 2) # y_t = g(h_t,y_{t-1},c)

            # 선형 레이어를 통해 score를 계산
            score = self.linear(concated.squeeze(0))

            # score에 softmax를 적용하여 확률 분포를 얻음
            softmaxed = F.log_softmax(score,1)

            # softmaxed를 decode 리스트에 추가
            decode.append(softmaxed)

            # softmaxed의 최대값 인덱스를 decoded에 저장
            decoded = softmaxed.max(1)[1]

            # decoded를 임베딩하여 embedded를 업데이트
            embedded = self.embedding(decoded).unsqueeze(1) # y_{t-1}
            
            # 훈련 단계에서만 드롭아웃 적용
            if is_training:
                embedded = self.dropout(embedded)

            # attention을 사용하여 다음 context 벡터를 계산
            context, alpha = self.Attention(hidden, encoder_outputs, encoder_maskings)

        # decode 리스트를 텐서로 변환
        scores = torch.cat(decode, 1)
        # scores의 크기를 변경
        return scores.view(inputs.size(0) * max_length, -1)

    # 디코딩 함수
    def decode(self, context, encoder_outputs):
        # 디코딩을 시작하는 심볼을 start_decode에 저장
        start_decode = Variable(torch.LongTensor([[2] * 1])).transpose(0, 1)
        # start_decode를 임베딩
        embedded = self.embedding(start_decode)
        # 은닉 상태를 초기화
        hidden = self.init_hidden(start_decode)

        decodes = []
        attentions = []
        decoded = embedded
        # decoded가 종료 심볼이 될 때까지 반복
        while decoded.data.tolist()[0] != 3: # </s>까지
            # GRU의 입력으로 embedded와 context를 연결한 것을 사용
            _, hidden = self.gru(torch.cat((embedded, context), 2), hidden) # h_t = f(h_{t-1},y_{t-1},c)
            # hidden과 context를 연결하여 concated를 생성
            concated = torch.cat((hidden, context.transpose(0, 1)), 2) # y_t = g(h_t,y_{t-1},c)
            # 선형 레이어를 통해 score를 계산
            score = self.linear(concated.squeeze(0))
            # score에 softmax를 적용하여 확률 분포를 얻음
            softmaxed = F.log_softmax(score,1)
            # softmaxed를 decodes 리스트에 추가
            decodes.append(softmaxed)
            # softmaxed의 최대값 인덱스를 decoded에 저장
            decoded = softmaxed.max(1)[1]
            # decoded를 임베딩하여 embedded를 업데이트
            embedded = self.embedding(decoded).unsqueeze(1) # y_{t-1}
            # attention을 사용하여 다음 context 벡터를 계산
            context, alpha = self.Attention(hidden, encoder_outputs,None)
            # alpha를 attentions 리스트에 추가
            attentions.append(alpha.squeeze(1))

        # decodes 리스트를 텐서로 변환하고 최대값 인덱스를 반환
        return torch.cat(decodes).max(1)[1], torch.cat(attentions)

In [ ]:
# 학습 함수 정의
def train_model(encoder, decoder, dataloader, encoder_optimizer, decoder_optimizer, criterion, num_epochs, max_length):
    encoder.train()
    decoder.train()

    for epoch in range(num_epochs):
        total_loss = 0
        for i, (src_seqs, trg_seqs) in enumerate(dataloader):
            # CUDA 사용 여부에 따라 데이터를 GPU로 이동
            src_seqs = src_seqs.squeeze(1)
            trg_seqs = trg_seqs.squeeze(1)
            if USE_CUDA:
                src_seqs = src_seqs.cuda()
                trg_seqs = trg_seqs.cuda()

            # 입력 시퀀스의 길이 계산
            input_lengths = torch.sum(src_seqs != 0, dim=1)

            # 인코더의 forward 호출
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()

            encoder_outputs, encoder_hidden = encoder(src_seqs, input_lengths)

            # 디코더의 첫 입력으로 <SOS> 토큰을 설정
            decoder_input = torch.LongTensor([[2]] * src_seqs.size(0))
            if USE_CUDA:
                decoder_input = decoder_input.cuda()

            # 인코더의 마지막 은닉 상태를 context로 사용
            context = encoder_hidden.unsqueeze(0)

            # 디코더의 forward 호출
            decoder_output = decoder(decoder_input, context, max_length, encoder_outputs, is_training=True)

            # 목표 시퀀스 길이에 맞게 타겟 시퀀스를 변경
            target = trg_seqs[:, :max_length].contiguous().view(-1)
            loss = criterion(decoder_output, target)
            loss.backward()

            # 가중치 업데이트
            encoder_optimizer.step()
            decoder_optimizer.step()

            total_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(dataloader):.4f}')

# 데이터 로드
dataset = MNTDataset("/home/pervinco/Datasets/fra-eng/fra.txt", 3, 25)
dataloader = dataset.get_dataloader(batch_size=64, shuffle=True, num_workers=2)

# 인코더 및 디코더 초기화
embedding_size = 256
hidden_size = 512
input_size_encoder = len(dataset.source2index)
input_size_decoder = len(dataset.target2index)
encoder = Encoder(input_size_encoder, embedding_size, hidden_size, n_layers=2, bidirec=True)
decoder = Decoder(input_size_decoder, embedding_size, hidden_size, n_layers=2)

# CUDA 사용 여부
if USE_CUDA:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss(ignore_index=dataset.target2index['<PAD>'])
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.001)

# 모델 학습
num_epochs = 20
max_length = 25
train_model(encoder, decoder, dataloader, encoder_optimizer, decoder_optimizer, criterion, num_epochs, max_length)
